# GconvGRU

Chaque modelName va correspondre à un graph à un certain temps

DynamicGraphTemporalSignal:  graph qui  peut évoluer (+ ou - de noeuds / edge)

## Céation du dataset depuis le fichier csv 

edge weight = abs(dst_execution_time - src_execution_time)

In [1]:
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal

class CustomDynamicGraphTemporalSignal(DynamicGraphTemporalSignal):
    def __init__(self, edge_indices, edge_weights, features, targets):
        super().__init__(edge_indices=edge_indices,edge_weights=edge_weights ,features=features, targets=targets)
        self.edge_weights = edge_weights

    def __iter__(self):
        for i in range(len(self.features)):
            yield type('Snapshot', (), {
                "x": self.features[i],
                "y": self.targets[i],
                "edge_index": self.edge_indices[i],
                "edge_weight": self.edge_weights[i]
            })

In [2]:
import pandas as pd
import numpy as np
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal

def encode_action_type(action_type):
    return {
        'timer': [1, 0, 0],
        'pub': [0, 1, 0],
        'sub': [0, 0, 1]
    }.get(action_type, [0, 0, 0])

def create_dataset(csv_path):
    df = pd.read_csv(csv_path)
    df = df.sort_values(by=["ModelName", "NodeName", "ActionOrder"])

    unique_topics = df["Topic"].dropna().unique()
    topic_to_id = {topic: i for i, topic in enumerate(unique_topics)}

    snapshots = []

    for model_name, group in df.groupby("ModelName"):
        node_ids = []
        features = []
        targets = []

        node_index_map = {} 
        node_exec_time = {}
        idx = 0

        # Construction des noeuds
        for _, row in group.iterrows():
            node_id = f'{row["NodeName"]}-{row["ActionType"]}'
            if node_id not in node_index_map:
                node_index_map[node_id] = idx
                idx += 1

                one_hot = encode_action_type(row["ActionType"])
                topic_id = topic_to_id.get(row["Topic"], -1)
                period = row["Value"] / 1000.0 if row["ActionType"] == "timer" else row["Value"]
                feature_vector = one_hot + [topic_id, period, row["ActionOrder"]]
                features.append(feature_vector)
                exec_time = row["ExecutionTime"]
                targets.append(exec_time)
                node_exec_time[node_id] = exec_time
                node_ids.append({
                    "id": node_id,
                    "NodeName": row["NodeName"],
                    "ActionType": row["ActionType"],
                    "Topic": row["Topic"]
                })

        num_nodes = len(node_index_map)
        x = np.array(features)
        y = np.array(targets)

        edge_list = []
        edge_weights = []

        # Arêtes : timer -> pub (même NodeName)
        timers = group[group["ActionType"] == "timer"]
        publishers = group[group["ActionType"] == "pub"]
        subscribers = group[group["ActionType"] == "sub"]

        for _, timer in timers.iterrows():
            for _, pub in publishers.iterrows():
                if timer["NodeName"] == pub["NodeName"]:
                    src = f'{timer["NodeName"]}-timer'
                    dst = f'{pub["NodeName"]}-pub'
                    if src in node_index_map and dst in node_index_map:
                        edge_list.append((node_index_map[src], node_index_map[dst]))
                        weight = abs(node_exec_time[dst] - node_exec_time[src])
                        edge_weights.append(weight)

        # Arêtes : pub -> sub (même topic)
        for _, pub in publishers.iterrows():
            for _, sub in subscribers.iterrows():
                if pub["Topic"] == sub["Topic"]:
                    src = f'{pub["NodeName"]}-pub'
                    dst = f'{sub["NodeName"]}-sub'
                    if src in node_index_map and dst in node_index_map:
                        edge_list.append((node_index_map[src], node_index_map[dst]))
                        weight = abs(node_exec_time[dst] - node_exec_time[src])
                        edge_weights.append(weight)

        if edge_list:

            edge_index = np.array(edge_list).T 
            edge_weight = np.array(edge_weights, dtype=np.float32)
   
        else:
            edge_index = np.empty((2, 0), dtype=np.int64)
            edge_weight = np.empty((0,), dtype=np.float32)
       

        snapshots.append((x, edge_index, edge_weight, y))

    features = [snap[0] for snap in snapshots]
    edge_indices = [snap[1] for snap in snapshots]
    edge_weights = [snap[2] for snap in snapshots]
    # print("****************************************")
    # print("edge_weights : ", edge_weights)
    targets = [snap[3] for snap in snapshots]
    for i, (ei, ew, f, t) in enumerate(zip(edge_indices, edge_weights, features, targets)):
        # print(f"Snapshot {i}: edge_index shape = {ei.shape}, edge_weight shape = {ew.shape if ew is not None else None}")
        if ew is not None and ei.shape[1] != ew.shape[0]:
            print(f"  Mismatch in snapshot {i} between edge_index and edge_weight lengths!")

    return CustomDynamicGraphTemporalSignal(
        edge_indices=edge_indices,
        edge_weights=edge_weights,
        features=features,
        targets=targets
    )





## Debug dataset

In [3]:
import networkx as nx
import matplotlib.pyplot as plt

def debug_dataset(dataset):
    for t, snapshot in enumerate(dataset):
        print(f"\n🕒 Snapshot {t}")
        print("Edge Index:")
        print(snapshot.edge_index)

        print("Node Features:")
        print(snapshot.x)
        print("Targets:")
        print(snapshot.y)

        # G = nx.DiGraph()
        # num_nodes = snapshot.x.shape[0]
        # for i in range(num_nodes):
        #     G.add_node(i, feature=snapshot.x[i], target=snapshot.y[i])

        # for src, dst in snapshot.edge_index.T:
        #     G.add_edge(src, dst)

        # pos = nx.spring_layout(G, seed=42)
        # labels = {i: f"{i}\n{snapshot.y[i]:.3f}" for i in range(num_nodes)}
        # nx.draw(G, pos, with_labels=True, labels=labels, node_color='skyblue', edge_color='gray')
        # plt.title(f"Graphe au snapshot {t}")
        # plt.show()


In [ ]:
dataset = create_dataset("../dataset/dataset.csv")
debug_dataset(dataset)


## Prédictions sans visualisation graphique 

In [ ]:
import torch
import numpy as np
import torch.nn.functional as F
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal
from torch_geometric_temporal.nn.recurrent import GConvGRU
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from torch.nn import L1Loss
mae_fn = L1Loss()

dataset = create_dataset("../dataset/dataset.csv")

all_features = np.vstack([snapshot.x for snapshot in dataset])
all_targets = np.concatenate([snapshot.y for snapshot in dataset]).reshape(-1, 1)

feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

feature_scaler.fit(all_features)
target_scaler.fit(all_targets)

# Modèle avec GConvGRU
class GConvGRUModel(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.recurrent = GConvGRU(
            in_channels=in_channels,
            out_channels=out_channels,
            K=2  
        )
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(out_channels, 1),
            torch.nn.Softplus()  
        )

    def forward(self, x, edge_index, edge_weight, h):
        h = self.recurrent(x, edge_index, edge_weight, h)
        out = self.linear(F.relu(h))
        return out, h

# Initialisation
model = GConvGRUModel(in_channels=6, out_channels=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()

import matplotlib.pyplot as plt

plt.hist(all_targets, bins=100)
plt.title("Distribution des temps d'exécution")
plt.xlabel("Execution Time (s)")
plt.ylabel("Count")
plt.grid(True)
plt.show()


# Entraînement
model.train()
for epoch in range(10):
    loss_total = 0
    mae_total = 0
    h = None
    for snapshot in dataset:
      
        x_norm = feature_scaler.transform(snapshot.x)
        y_norm = target_scaler.transform(snapshot.y.reshape(-1, 1))

        x = torch.FloatTensor(x_norm)
        y = torch.FloatTensor(y_norm)

        edge_index = torch.LongTensor(snapshot.edge_index.T if snapshot.edge_index.shape[0] != 2 else snapshot.edge_index)
        edge_weight = torch.FloatTensor(snapshot.edge_weight) if snapshot.edge_weight is not None else torch.ones(edge_index.shape[1])


        if h is None or h.size(0) != x.size(0):
            h = torch.zeros(x.size(0), model.recurrent.out_channels)


        optimizer.zero_grad()
        out, h = model(x, edge_index, edge_weight, h)
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
        if h is not None:
            h = h.detach()
        loss_total += loss.item()
        mae_total += mae_fn(out, y).item()
    print(f"Epoch {epoch+1}, Loss: {loss_total:.4f}, MAE: {mae_total:.4f}")

# Prédiction
print("\n Predictions :")
model.eval()
h = None
#for snapshot in dataset:
for t, snapshot in enumerate(dataset):
    x_norm = feature_scaler.transform(snapshot.x)
    x = torch.FloatTensor(x_norm)
    # y = torch.FloatTensor(snapshot.y)
    y_norm = target_scaler.transform(snapshot.y.reshape(-1, 1))
    y = torch.FloatTensor(y_norm)

 
    edge_index = torch.LongTensor(snapshot.edge_index.T if snapshot.edge_index.shape[0] != 2 else snapshot.edge_index)
    edge_weight = torch.FloatTensor(snapshot.edge_weight) if snapshot.edge_weight is not None else torch.ones(edge_index.shape[1])


    if h is None or h.size(0) != x.size(0):
        h = torch.zeros(x.size(0), model.recurrent.out_channels)


    out, h = model(x, edge_index, edge_weight, h)
    out_norm = out.view(-1).detach().numpy().reshape(-1, 1)
    out_original = target_scaler.inverse_transform(out_norm).flatten()
    y_norm = y.view(-1).detach().numpy().reshape(-1, 1)
    real = target_scaler.inverse_transform(y_norm).flatten()
    print(f"Prédiction (dénormalisée) : {out_original}")
    print(f"Réel                   : {real}")
    for i, (p, r) in enumerate(zip(out_original, real)):
        print(f"Noeud {i} -> Prédiction: {p:.3f}, Réel: {r:.3f}")
        print()
    # plt.figure(figsize=(6,4))
    # nodes = np.arange(len(out_original))
    # plt.plot(nodes, real, 'o-', label='Valeurs réelles')
    # plt.plot(nodes, out_original, 'x--', label='Prédictions')
    # plt.title(f"Comparaison prédictions vs valeurs réelles (snapshot {t})")
    # plt.xlabel("Noeud")
    # plt.ylabel("Valeur")
    # plt.legend()
    # plt.grid(True)
    # plt.show()


## avec matplotlib (visualisation graphique données réelles VS prédictions)

In [ ]:
import torch
import numpy as np
import torch.nn.functional as F
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal
from torch_geometric_temporal.nn.recurrent import GConvGRU
import matplotlib.pyplot as plt


from torch.nn import L1Loss
mae_fn = L1Loss()

# Création du dataset dynamique
edge_indices = [
    np.array([[0, 1], [1, 2]]).T,
    np.array([[0, 2], [2, 1]]).T,
    np.array([[1, 0], [2, 0]]).T,
]

edge_weights = [
    np.array([1.0, 2.0]),
    np.array([0.5, 1.5]),
    np.array([2.0, 1.0])
]

features = [
    np.array([[1, 0], [0, 1], [1, 1]]),
    np.array([[2, 0], [1, 1], [0, 1]]),
    np.array([[1, 1], [2, 1], [1, 0]]),
]

targets = [
    np.array([0.1, 0.2, 0.3]),
    np.array([0.2, 0.1, 0.4]),
    np.array([0.3, 0.2, 0.5]),
]

dataset = DynamicGraphTemporalSignal(
    edge_indices=edge_indices,
    edge_weights=edge_weights,
    features=features,
    targets=targets,
)

# Modèle avec GConvGRU
class GConvGRUModel(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.recurrent = GConvGRU(
            in_channels=in_channels,
            out_channels=out_channels,
            K=2 
        )
        self.linear = torch.nn.Linear(out_channels, 1)

    def forward(self, x, edge_index, edge_weight, h):
        h = self.recurrent(x, edge_index, edge_weight, h)
        out = self.linear(F.relu(h))
        return out, h

# Initialisation
model = GConvGRUModel(in_channels=2, out_channels=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()

# Entraînement
model.train()
for epoch in range(300):
    loss_total = 0
    mae_total = 0
    h = None
    for snapshot in dataset:
        x = torch.FloatTensor(snapshot.x)
        y = torch.FloatTensor(snapshot.y).view(-1, 1)
        edge_index = torch.LongTensor(snapshot.edge_index.T)
        edge_weight = (
            torch.FloatTensor(snapshot.edge_weight)
            if snapshot.edge_weight is not None
            else torch.ones(edge_index.shape[1])
        )

        if h is None:
            h = torch.zeros(x.size(0), 4)  

        optimizer.zero_grad()
        out, h = model(x, edge_index, edge_weight, h)
        loss = loss_fn(out, y)
        loss.backward(retain_graph=True )
        optimizer.step()
        if h is not None:
            h = h.detach()
        loss_total += loss.item()
        mae_total += mae_fn(out, y).item()
    print(f"Epoch {epoch+1}, Loss: {loss_total:.4f}, MAE: {mae_total:.4f}")

# Prédiction
print("\n📈 Predictions :")
model.eval()
h = None
for t, snapshot in enumerate(dataset):
    x = torch.FloatTensor(snapshot.x)
    y = torch.FloatTensor(snapshot.y)
    edge_index = torch.LongTensor(snapshot.edge_index)
    edge_weight = (
        torch.FloatTensor(snapshot.edge_weight)
        if snapshot.edge_weight is not None
        else torch.ones(edge_index.shape[1])
    )
    out, h = model(x, edge_index, edge_weight, h)
    pred = out.view(-1).detach().numpy()
    real = y.numpy()

    plt.figure(figsize=(6,4))
    nodes = np.arange(len(pred))
    plt.plot(nodes, real, 'o-', label='Valeurs réelles')
    plt.plot(nodes, pred, 'x--', label='Prédictions')
    plt.title(f"Comparaison prédictions vs valeurs réelles (snapshot {t})")
    plt.xlabel("Noeud")
    plt.ylabel("Valeur")
    plt.legend()
    plt.grid(True)
    plt.show()

